# Linear Solver Performance: Constant Coefficient Poisson, Single Core

### Part 2, Evaluation

In [1]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using System.IO;
using BoSSS.Application.SipPoisson;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [ ]:
wmg.Init("LinslvPerfSer");
wmg.SetNameBasedSessionJobControlCorrelation();

In [ ]:
// Manually add database (e.g. when accessing the database of user `JenkinsCI` as some other, regular guy)
//var tempDb = OpenOrCreateDatabase(@"\\fdygitrunner\ValidationTests\LinslvPerf_ConstPoissonMpi1");

In [ ]:
databases

In [ ]:
wmg.Sessions

In [ ]:
var SI = wmg.Sessions[36];
SI

In [ ]:
SI.KeysAndQueries

In [ ]:
SI.GetControl().GetSolverType().Name

In [ ]:
//var s = wmg.Sessions.Single(s => s.Name.Contains("J20480_k3_exp_Kcycle_schwarz"));
//s.DeployPath

## Create Table for Post-Processing

The timeing information that is requires for this study is not pesent in the default session table.
Instead, it must be extracted from the *session profiling*.

To obtain timing-measurements, the instrumentation generated by certain `BlockTrace`-blocks within the `UniSolver` class is extracted and added to the data table:

In [ ]:
// evaluators to add additional columns to the session table
static class AddCols {
    static int GetSolType(ISessionInfo SI) {
        if(SI.GetControl().GetSolverType().Name.Contains("XNSE"))
            return 3;
        if(SI.GetControl().GetSolverType().Name.Contains("XdgPoisson3Main"))
            return 2;
        return 1;
    }
    
    
    static public object XdgMatrixAssembly_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        ilPSP.Tracing.MethodCallRecord nd;
        if(GetSolType(SI) == 3)
            nd = mcr.FindChildren("*.XdgTimestepping.ComputeOperatorMatrix").Single();
        else 
            nd  = mcr.FindChildren("MatrixAssembly").Single();
        //var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Aggregation_basis_init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var nd  = mcr.FindChildren("Aggregation_basis_init").Single();
        //var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_Init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var nd  = mcr.FindChildren("Solver_Init").Single();
        //var nd  = ndS.ElementAt(0);
        //Console.WriteLine("Number of nodes: " + ndS.Count() + " cc " + nd.CallCount );
        return nd.TimeSpentInMethod.TotalSeconds / nd.CallCount;
    }
    static public object Solver_Run_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var nd  = mcr.FindChildren("Solver_Run").Single();
        //var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_InitAndRun_time(ISessionInfo SI) {
        double agitime = (double) Aggregation_basis_init_time(SI);
        double initime = (double) Solver_Init_time(SI);
        double runtime = (double) Solver_Run_time(SI);
        
        return agitime + initime + runtime;
    }
    static public object Solver_TimePerIter(ISessionInfo SI) {
        double runtime = (double) Solver_Run_time(SI);
        double NoOfItr = Convert.ToDouble(SI.KeysAndQueries["NoIter"]);
        return runtime/NoOfItr;
    }
    static public object NoOfCores(ISessionInfo SI){
        return SI.GetProfiling().Length;
    }
    static public object ComputeNodeName(ISessionInfo SI) {
        return SI.ComputeNodeNames.First();
    }
    
    static public object DOFs(ISessionInfo SI) {
        int NoOfItr = Convert.ToInt32(SI.KeysAndQueries["NoIter"]);
        return NoOfItr;
    }
    
    static public object DGdegree(ISessionInfo SI) {
        switch(GetSolType(SI)) {
            case 3:
            return SI.KeysAndQueries.Single(kv => kv.Key.Contains("DGdegree:Velocity")).Value;
            case 2:
            return SI.KeysAndQueries.Single(kv => kv.Key == "DGdegree:u").Value;    
            case 1:
            return SI.KeysAndQueries.Single(kv => kv.Key == "DGdegree:T").Value;
        }
        throw new ArgumentException();
    }
}

In [ ]:
wmg.AdditionalSessionTableColums.Clear();
wmg.AdditionalSessionTableColums.Add("MatrixAssembly", AddCols.XdgMatrixAssembly_time);
wmg.AdditionalSessionTableColums.Add("Aggregation_basis_init_time", AddCols.Aggregation_basis_init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Init_time", AddCols.Solver_Init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Run_time", AddCols.Solver_Run_time);
wmg.AdditionalSessionTableColums.Add("Solver_InitAndRun_time", AddCols.Solver_InitAndRun_time);
wmg.AdditionalSessionTableColums.Add("Solver_TimePerIter", AddCols.Solver_TimePerIter);
wmg.AdditionalSessionTableColums.Add("NoOfCores", AddCols.NoOfCores);
wmg.AdditionalSessionTableColums.Add("ComputeNodeName", AddCols.ComputeNodeName);
wmg.AdditionalSessionTableColums.Add("DGdegree", AddCols.DGdegree);
wmg.AdditionalSessionTableColums.Add("DOFs", AddCols.DOFs);

In [ ]:
var SessTab = wmg.SessionTable;

//// The Session column can't be serialized,
//// we have to remove it
//List<string> AllCols = FullSessTab.GetColumnNames().ToList();
//AllCols.Remove("Session");
//FullSessTab = FullSessTab.ExtractColumns(AllCols.ToArray());

In [ ]:
//wmg.Sessions[0].KeysAndQueries["NoIter"].GetType()

In [ ]:
//wmg.Sessions[0].Name

In [ ]:
//SessTab.GetColumn("LinearSolver.Name")

In [ ]:
//SessTab.GetColumnNames().ToConcatString("", "; ", "")

Select those columns which are of interest:

In [ ]:
var SubTab = SessTab.ExtractColumns(
    "SessionName","DGdegree", "Grid:NoOfCells", "LinearSolver.Name", "LinearSolver.Shortname", "DOFs", "MatrixAssembly",
    "Aggregation_basis_init_time", "Solver_Init_time", "Solver_Run_time", "Solver_InitAndRun_time", "NoIter", 
    "Solver_TimePerIter", "ComputeNodeName");

In [ ]:
// Filename
var now         = DateTime.Now;
string docName  = wmg.CurrentProject + "_" + now.Year + "-" + now.Month + "-" + now.Day;
//SubTab.SaveToFile(docName + ".json");
//SubTab.ToCSVFile(docName + ".csv");

In [2]:
var SubTab = TableExtensions.LoadFromFile("temp.json");

In [3]:
//SubTab.Print();

## Vizualisation of Results

The following data is available:

In [4]:
SubTab.GetColumnNames()

index,value
0,SessionName
1,DGdegree
2,Grid:NoOfCells
3,LinearSolver.Name
4,LinearSolver.Shortname
5,DOFs
6,MatrixAssembly
7,Aggregation_basis_init_time
8,Solver_Init_time
9,Solver_Run_time


Available DG degrees:

In [5]:
var DGdegrees = SubTab.GetColumn<int>("DGdegree").ToSet().OrderBy(s => s).ToArray();
DGdegrees

index,value
0,2
1,3
2,5


All used solvers:

In [6]:
SubTab.GetColumn<string>("LinearSolver.Shortname").ToSet()

index,value
0,PARDISO
1,OrthoMG w Add Swz
2,GMRES w p2G


In [7]:
const string Pardiso = "PARDISO";
const string OrmgSwz = "OrthoMG w Add Swz";
const string GmsrP2g = "GMRES w p2G";
const string OrmgILU = "OrthoMG w ILU";

Used Cases:

In [8]:
const string Poisson = "SIP_Poisson";
const string XPoisson = "XdgPoisson";
const string Stokes2D = "BottiPietroStokes2D";
const string Stokes3D = "BottiPietroStokes3D";
const string XStokes = "XdgStokes";
string[] AllCases = new string[] { Poisson, XPoisson, Stokes2D, Stokes3D, XStokes };

In [9]:
string ExtractCase(string sessionName) {
    return AllCases.Single(caseName => sessionName.Contains(caseName));
}

### Macro Routine for Multiplot

The following routine combines the plotting code which is common for all sub-plot in this evaluation;
only the y-axis needs to be specified. 

In [10]:
PlotFormat SlvCode2Pltfmt(string solver_name, string caseName) { 
    var Fmt = new PlotFormat();
    switch(solver_name) {
        case Pardiso: 
            Fmt.PointType = PointTypes.OpenCircle;
            Fmt.DashType  = DashTypes.Dotted;
            break;
        case GmsrP2g: 
            Fmt.PointType = PointTypes.Box;
            break;
        case OrmgSwz: 
            Fmt.PointType = PointTypes.LowerTriangle;
            break;
        case OrmgILU:
            Fmt.PointType = PointTypes.Diamond;
            break;
        default:
            Console.WriteLine("unknown: " + solver_name); 
            break;
    } 
    //Console.WriteLine("name is: " + solver_name); 
    Fmt.PointSize = 0.85;
    Fmt.LineWidth = 2;    
    Fmt.Style     = Styles.LinesPoints;
    
    if(caseName.Contains(Poisson))
        Fmt.LineColor = LineColors.Green;
    else if(caseName.Contains(XPoisson))
        Fmt.LineColor = LineColors.Blue;
    else if(caseName.Contains(Stokes2D))
        Fmt.LineColor = LineColors.Orange;
    else if(caseName.Contains(XStokes))
        Fmt.LineColor = LineColors.Red;
    

    return Fmt;
}

In [11]:
void AssertSlope(Plot2Ddata plot, string caseName, string solverName, double allowedSlope) {
   NUnit.Framework.Assert.LessOrEqual(
      plot.Regression().Single(tt =>  tt.Key.Contains(caseName) && tt.Key.Contains(solverName)).Value,
      1.1,
      $"{solverName} scaling for {caseName} exceeds limit");
}

In [12]:
Plot2Ddata[,] PlotSolverBehave(string Yname, bool LogY, double yMin, double yMax) {

int rows    = DGdegrees.Length;
int columns = 1;
string[] ignore_solvers = {};
Plot2Ddata[,] multiplots = new Plot2Ddata[rows,columns];
int pDegree = 0;
for(int iRow = 0; iRow < rows; iRow++) {
for(int iCol = 0; iCol < columns; iCol++) {
    
    if(pDegree > rows*columns-1)
        continue;
    int tmpDG = -1;
    if(pDegree < DGdegrees.Length)
        tmpDG = DGdegrees[pDegree];
    
    //Create Graphs
    multiplots[iRow,iCol] = SubTab.ToPlot("DOFs", Yname, // column for x- and y
       delegate (int iTabRow, 
                 IDictionary<string, object> Row, 
                 out string Nmn, 
                 out PlotFormat Fmt) { 
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           // PlotRowSelector:
           // selects, which table row goes to which graph,
           // and the respective color  
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           int k = Convert.ToInt32(Row["DGdegree"]);
           if(k != tmpDG) {
                // degree does not match -> not in this plot
                Nmn = null;
                Fmt = null;
                return;
           }
 
           string solver_name = (string) (Row["LinearSolver.Shortname"]);
           //ignore the solvers specified in ingore_solvers
           foreach(string sc in ignore_solvers){
               if(solver_name == sc){
                   System.Console.WriteLine("skipped");
                   Nmn = null;
                   Fmt = null;
               return;
               }
           }
           
           
           //process the other solvers
           string caseName = ExtractCase( (string) Row["SessionName"]);
           Fmt = SlvCode2Pltfmt(solver_name, caseName);
           Nmn = solver_name + "/" + caseName;
       });
    
       double[] dof = new[]{1e3,1e6};
       double[] linT = dof.Select(x => x*0.001).ToArray();
       var linP = new Plot2Ddata.XYvalues("linear", dof, linT);
       linP.Format.FromString("- black");
       ArrayTools.AddToArray(linP, ref multiplots[iRow,iCol].dataGroups);
 
       //all about axis
       string Title = string.Format("$k = {0}$", tmpDG);
       multiplots[iRow,iCol].Ylabel = Title;
       multiplots[iRow,iCol].LogX = true;
       multiplots[iRow,iCol].LogY = LogY;
    
       //specify range of axis
       multiplots[iRow,iCol].YrangeMin = yMin;
       multiplots[iRow,iCol].YrangeMax = yMax;
       multiplots[iRow,iCol].XrangeMin = 1e2;
       multiplots[iRow,iCol].XrangeMax = 1e7;
    
       //multiplots[iRow,iCol].Y2rangeMin = 1e-3;
       //multiplots[iRow,iCol].Y2rangeMax = 1e+4;
       //multiplots[iRow,iCol].X2rangeMin = 1e2;
       //multiplots[iRow,iCol].X2rangeMax = 1e7;
    
       //spacing around plots
       multiplots[iRow,iCol].ShowLegend = false;
       multiplots[iRow,iCol].tmargin = 0;
       multiplots[iRow,iCol].bmargin = 2;
       multiplots[iRow,iCol].lmargin = 5;
       multiplots[iRow,iCol].rmargin = 25;
       multiplots[iRow,iCol].ShowXtics = false;

       //I am legend ...
       if(iRow == 0) {
          multiplots[iRow,iCol].ShowLegend = true;
          //multiplots[iRow,iCol].LegendAlignment = new string[]{"o", "r", "t" };
          multiplots[iRow,iCol].LegendFont = 12;
          multiplots[iRow,iCol].Legend_maxrows = 100;
           multiplots[iRow,iCol].LegendPosition = new double[] { 4e9, 1e3 };
          //multiplots[iRow,iCol].LegendSwap  = true;
       }
       //and i am special ...
       if(iRow == rows - 1)
           multiplots[iRow,iCol].ShowXtics = true;
    pDegree++;
}                        
}
//multiplots.PlotCairolatex().WriteMinimalCompileableExample("latex/solvers.tex");
//multiplots.AddDummyPlotsForLegend(3,0);
return multiplots;
}

### Total Runtime

One would expect **linear scaling** with grid resolution; it may scale nonlinear with DG polynomial order.

In [13]:
var multiplots = PlotSolverBehave("Solver_InitAndRun_time", true, 1e-2, 1e+4);
//multiplots.PlotCairolatex().PlotNow()
//multiplots.AddDummyPlotsForLegend(3,0);
multiplots.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 GMRES w p2G/SIP P oisson 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 OrthoMG w Add Swz/SIP P oisson 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 PARDISO/SIP P oisson 
 
 
 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 $10 7 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 
 
 
 linear

In [21]:
var legend = new Plot2Ddata();
legend.AddDataGroup("Pardiso",new double[] {1,2},new double[] {1,2}, "-og");
legend.ShowXtics = false;
legend.ShowXtics = false;
//legend.XrangeMin = 3;
//legend.XrangeMax = 3.1;

//legend.AddGnuplotCommand("set border 0");
//legend.AddGnuplotCommand("set style line 101 lc rgb '#808080' lt 1 lw 1");
legend.AddGnuplotCommand("unset xlabel");
legend.AddGnuplotCommand("unset ylabel");
legend.AddGnuplotCommand("set format x ''");
legend.AddGnuplotCommand("set format y ''");
legend.AddGnuplotCommand("set tics scale 0");

legend.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Pardiso 
 
 
 Pardiso

#### Regression and Error checking

In [ ]:
multiplots[0,0].Regression()

In [ ]:
multiplots[1,0].Regression()

In [ ]:
multiplots[2,0].Regression()

In [ ]:
AssertSlope(multiplots[0,0], Poisson, GmsrP2g, 1.1);
AssertSlope(multiplots[1,0], Poisson, GmsrP2g, 1.1);
AssertSlope(multiplots[2,0], Poisson, GmsrP2g, 1.1);

Experimental: Latex Preview:

In [ ]:
//var multiplots = PlotSolverBehave("Solver_InitAndRun_time", true, 1e-2, 1e+4);
//multiplots.PlotCairolatex().PlotNow()

### Number of Iterations

For the iterative solvers, obe would expect **almost constant number of solver iterations**, 
independent of grid resolution. It may, however depend in nonlinar fashion on the DG polynomial order.

In [ ]:
var multiplots = PlotSolverBehave("NoIter", false, 0, 140);
//multiplots.PlotCairolatex().WriteMinimalCompileableExample("latex/solvers.tex");
//multiplots.AddDummyPlotsForLegend(3,0);
multiplots.PlotNow()

In [ ]:
multiplots[0,0].dataGroups

In [ ]:
multiplots[1,0].dataGroups

In [ ]:
multiplots[2,0].dataGroups

### Runtime per Iteration

One would expect **linear scaling** with grid resolution; it may scale nonlinear with DG polynomial order.

In [ ]:
var multiplots = PlotSolverBehave("Solver_TimePerIter", true, 1e-2, 1e+3);
//multiplots.PlotCairolatex().WriteMinimalCompileableExample("latex/solvers.tex");
//multiplots.AddDummyPlotsForLegend(3,0);
multiplots.PlotNow()

In [ ]:
multiplots[0,0].Regression()

In [ ]:
multiplots[1,0].Regression()

In [ ]:
multiplots[2,0].Regression()

In [ ]:
AssertSlope(multiplots[0,0], Poisson, GmsrP2g, 1.1);
AssertSlope(multiplots[1,0], Poisson, GmsrP2g, 1.1);
AssertSlope(multiplots[2,0], Poisson, GmsrP2g, 1.1);

## Appendix: Table Form of Extracted Data

In [ ]:
SubTab.Print()